In [1]:
import sys
sys.path.insert(1, '../model')
import model

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.ticker import ScalarFormatter
import optuna
optuna.logging.disable_default_handler()
import pandas as pd
pd.plotting.register_matplotlib_converters()
import seaborn as sns
from scipy.integrate import solve_ivp

In [3]:
raw = pd.read_csv("../data/covid_19_data.csv")

In [4]:
raw

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,4,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,5,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
14806,14807,04/13/2020,Wyoming,US,2020-04-13 23:15:42,275.0,1.0,0.0
14807,14808,04/13/2020,Xinjiang,Mainland China,2020-04-13 23:15:42,76.0,3.0,73.0
14808,14809,04/13/2020,Yukon,Canada,2020-04-13 23:15:42,8.0,0.0,0.0
14809,14810,04/13/2020,Yunnan,Mainland China,2020-04-13 23:15:42,184.0,2.0,174.0


In [5]:
data_cols = ["Infected", "Deaths", "Recovered"]
data_cols_all = ["Confirmed", "Infected", "Deaths", "Recovered"]
df = raw.rename({"ObservationDate": "Date", "Province/State": "Province"}, axis=1)
df["Date"] = pd.to_datetime(df["Date"])
df["Country"] = df["Country/Region"]
df.loc[df["Country"] == "Diamond Princess", ["Country", "Province"]] = ["Others", "Diamond Princess"]
df["Infected"] = df["Confirmed"] - df["Deaths"] - df["Recovered"]
df[data_cols_all] = df[data_cols_all].astype(np.int64)
ncov_df_ungrouped = df.loc[:, ["Date", "Country", "Province", *data_cols_all]]
ncov_df_ungrouped.tail()

,Date,Country,Province,Confirmed,Infected,Deaths,Recovered
14806,2020-04-13,US,Wyoming,275,274,1,0
14807,2020-04-13,Mainland China,Xinjiang,76,0,3,73
14808,2020-04-13,Canada,Yukon,8,8,0,0
14809,2020-04-13,Mainland China,Yunnan,184,8,2,174
14810,2020-04-13,Mainland China,Zhejiang,1267,27,1,1239


In [6]:
df_spain = ncov_df_ungrouped.loc[ncov_df_ungrouped["Country"] == "Spain", :].groupby("Date").sum()
df_spain

,Confirmed,Infected,Deaths,Recovered
Date,,,,
2020-02-01,1,1,0,0
2020-02-02,1,1,0,0
2020-02-03,1,1,0,0
2020-02-04,1,1,0,0
2020-02-05,1,1,0,0
...,...,...,...,...
2020-04-09,153222,85610,15447,52165
2020-04-10,158273,86524,16081,55668
2020-04-11,163027,87312,16606,59109


In [7]:
#if everything is 0 it means that there is no valid information yet
df_test = pd.read_csv("../data/full-list-total-tests-for-covid-19.csv")
df_test.rename({"Cumulative total tests": "TotalTests"}, axis=1,inplace=True)
df_test["Date"] = pd.to_datetime(df_test["Date"])
df_test_spain = df_test[df_test['Entity'] == 'Spain']
df_test_spain.tail()



,Entity,Code,Date,TotalTests


In [8]:
result = pd.merge(df_spain,
                 df_test_spain[['Date','TotalTests']],
                 on='Date',
                 how = 'left')
from numpy import isnan
result.loc[0, ('TotalTests')] = 0 if isnan(result['TotalTests'][0]) else result['TotalTests'][0]
result['TotalTests'].interpolate(method='linear',inplace=True,limit_direction='both')
result['TotalTests'] = result['TotalTests'].astype(int)
result

,Date,Confirmed,Infected,Deaths,Recovered,TotalTests
0,2020-02-01,1,1,0,0,0
1,2020-02-02,1,1,0,0,0
2,2020-02-03,1,1,0,0,0
3,2020-02-04,1,1,0,0,0
4,2020-02-05,1,1,0,0,0
...,...,...,...,...,...,...
68,2020-04-09,153222,85610,15447,52165,0
69,2020-04-10,158273,86524,16081,55668,0
70,2020-04-11,163027,87312,16606,59109,0
71,2020-04-12,166831,87231,17209,62391,0
